# Nama : Peno
# Nim : 221220095
# Kelas : 29 Informatika

In [40]:
from collections import defaultdict
import re
import pandas as pd
import json

df = pd.read_csv('berita_olahraga_scraped.csv')
print(df)
df.head()


      No                                              Judul  \
0      1  Jonatan Christie Antara Program Latihan Hingga...   
1      2  Blak-blakan Jonatan Christie Ungkap Alasan Mun...   
2      3  Ambisi Jonatan Christie: Olimpiade Los Angeles...   
3      4  Singapore Open Jadi Debut Jonatan Christie seb...   
4      5  Soal Gregoria, Taufik Hidayat: Kita Doakan Seh...   
..   ...                                                ...   
330  331  Hasil Milan Vs Monza 2-0: Rossoneri Tutup Seri...   
331  332  Antonio Conte Antar Napoli Juara, Jadi Bukti K...   
332  333  Daftar Penghargaan Liga Italia 2024-2025: Scot...   
333  334  Napoli Juara Liga Italia: Keajaiban Conte, Mel...   
334  335  Skor Como Vs Inter Milan 0-2: Nerazzurri Menan...   

                                                   Isi  \
0    Jonatan Christiemengaku telah mempersiapkan se...   
1    Jonatan Christie memutuskan mundur dari Pelatn...   
2    Jonatan Christiememang sudah keluar dari Pelat...   
3    Jonata

,No,Judul,Isi,Url,Label
0,1,Jonatan Christie Antara Program Latihan Hingga...,Jonatan Christiemengaku telah mempersiapkan se...,https://sport.detik.com/raket/d-7919041/jonata...,Non-Sepak Bola
1,2,Blak-blakan Jonatan Christie Ungkap Alasan Mun...,Jonatan Christie memutuskan mundur dari Pelatn...,https://www.detik.com/jogja/berita/d-7918868/b...,Non-Sepak Bola
2,3,Ambisi Jonatan Christie: Olimpiade Los Angeles...,Jonatan Christiememang sudah keluar dari Pelat...,https://sport.detik.com/raket/d-7917849/ambisi...,Non-Sepak Bola
3,4,Singapore Open Jadi Debut Jonatan Christie seb...,Jonatan Christiememastikan debutnya sebagai pe...,https://sport.detik.com/raket/d-7917140/singap...,Non-Sepak Bola
4,5,"Soal Gregoria, Taufik Hidayat: Kita Doakan Seh...",Pebulutangkis putri IndonesiaGregoria Mariska ...,https://sport.detik.com/raket/d-7917117/soal-g...,Non-Sepak Bola


In [41]:
print("📄 Jumlah dokumen:", len(df))

📄 Jumlah dokumen: 335


In [42]:
def tokenize(text):
    return re.findall(r'\b\w+\b', text.lower())
# Contoh tokenisasi dokumen ke-1
df['Tokens'] = df['Isi'].apply(tokenize)
print(df[['Isi', 'Tokens']].head())

                                                 Isi  \
0  Jonatan Christiemengaku telah mempersiapkan se...   
1  Jonatan Christie memutuskan mundur dari Pelatn...   
2  Jonatan Christiememang sudah keluar dari Pelat...   
3  Jonatan Christiememastikan debutnya sebagai pe...   
4  Pebulutangkis putri IndonesiaGregoria Mariska ...   

                                              Tokens  
0  [jonatan, christiemengaku, telah, mempersiapka...  
1  [jonatan, christie, memutuskan, mundur, dari, ...  
2  [jonatan, christiememang, sudah, keluar, dari,...  
3  [jonatan, christiememastikan, debutnya, sebaga...  
4  [pebulutangkis, putri, indonesiagregoria, mari...  


In [43]:
def build_inverted_index(docs):
    index = defaultdict(set)
    for doc_id, text in docs.items():
        for word in tokenize(text):
            index[word].add(doc_id)
    return index

dokumen = df['Isi'].to_dict()
inverted_index = build_inverted_index(dokumen)
universal_set = set(dokumen.keys())
# Tampilkan inverted index
for i, (term, posting) in enumerate(sorted(inverted_index.items())):
    print(f"{term:15} → {sorted(posting)}")

0               → [11, 12, 13, 17, 22, 30, 33, 34, 43, 61, 63, 76, 100, 103, 108, 110, 114, 116, 117, 118, 119, 120, 122, 125, 130, 132, 133, 134, 136, 137, 138, 143, 154, 157, 173, 180, 181, 183, 185, 186, 187, 188, 189, 190, 196, 220, 222, 224, 227, 242, 249, 256, 257, 258, 260, 269, 271, 276, 277, 279, 284, 286, 299, 306, 312, 313, 314, 320, 322, 324, 328, 329, 330, 332, 333, 334]
00              → [22, 32, 61, 78, 79, 105, 110, 111, 112, 113, 114, 115, 118, 119, 120, 121, 123, 124, 125, 126, 127, 128, 154, 179, 192, 209, 213, 215, 216, 217, 225, 226, 243, 310]
000             → [10, 14, 22, 25, 31, 50, 71, 76, 175, 209, 276, 328]
01              → [125, 154]
013             → [206]
017             → [17]
0187            → [17]
02              → [59, 161, 277, 310]
020             → [25]
03              → [154, 207]
030             → [206]
04              → [17, 277]
05              → [12, 187, 188, 234, 277, 309, 329]
059             → [297]
06              → [59, 161, 187, 188, 20

In [44]:
def boolean_search(query):
    tokens = tokenize(query)
    if not tokens:
        return set()
    
    def term(pos):
        neg = False
        if tokens[pos] == "not":
            neg = True
            pos += 1
            if pos >= len(tokens):
                raise ValueError("Operator NOT tidak diikuti term.")
        word = tokens[pos]
        postings = set(inverted_index.get(word, set()))
        if neg:
            postings = universal_set - postings
        return postings, pos + 1
    result, pos = term(0)
    while pos < len(tokens):
        op = tokens[pos]
        if op not in ("and", "or"):
            raise ValueError(f"Operator tidak dikenali: {op}")
        pos += 1
        rhs, pos = term(pos)
        if op == "and":
            result &= rhs
        else:
            result |= rhs
    return result

results = {}


In [45]:
queries = [
    "Anthony and MU",
    "Badminton or Basket",
    "Laliga and not Messi",
    "dari and not mesin",
    "not Inggris",
    "Milan and Kalah"
]
for q in queries:
    try:
        hasil = boolean_search(q)
        results[q] = sorted(hasil)
        print(f"Query: '{q}' → {sorted(hasil)}")
    except (IndexError, ValueError) as e:
        results[q] = sorted(hasil)
        print(f"Query: '{q}' → Error: {e}")

Query: 'Anthony and MU' → [155, 176]
Query: 'Badminton or Basket' → [0, 33, 34, 36, 41, 46, 48, 51, 52, 54, 55, 56, 57, 58, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 250]
Query: 'Laliga and not Messi' → [242, 244, 246, 276, 278, 282, 283, 285, 290, 291, 292]
Query: 'dari and not mesin' → [0, 1, 2, 3, 4, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 18, 19, 20, 21, 22, 23, 25, 27, 28, 29, 30, 31, 32, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 49, 50, 51, 52, 53, 54, 56, 57, 58, 60, 61, 62, 63, 65, 66, 67, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 102, 103, 104, 106, 107, 108, 111, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 125, 126, 127, 128, 129, 130, 131, 132, 134, 135, 136, 137, 138, 139, 140, 143, 144, 145, 146, 148, 149, 151, 152, 154, 155, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168,

In [46]:
# Menyimpan hasil pencarian ke dalam file JSON
with open('output_results.json', 'w') as json_file:
    json.dump(results, json_file, indent=4)

print("Hasil pencarian telah disimpan ke 'output_results.json'.")

Hasil pencarian telah disimpan ke 'output_results.json'.
